# Robot arm kinematic control

This page introduce the concept of controlling the end-effector of a robot when its joint are controlled with velocity set-points.

## Importing Librairies

In [ ]:
# Loading a robotic toolbox
!git clone https://github.com/SherbyRobotics/pyro
import sys
sys.path.append('/content/pyro')
import pyro

# Generic python tools
import numpy as np
from IPython import display

# Defining the robot arm

Here we load a already defined class from the library including the dynamic equations defining the robot arm behavior.



In [ ]:
from pyro.dynamic import manipulator

# Dynamic model (inputs are motor torques)
torque_controlled_robot    = manipulator.TwoLinkManipulator()

torque_controlled_robot.l1 = 3.0 # length of first rigid link
torque_controlled_robot.l2 = 3.0 # length of second rigid link
torque_controlled_robot.l_domain = 10

# Kinematic only model (inputs are motor velocities)
speed_controlled_robot  = manipulator.SpeedControlledManipulator.from_manipulator( torque_controlled_robot )

robot = speed_controlled_robot # For this exercise, we will only use the kinematic model

## Showing the robot behavior with constant velocity inputs

Here we run a simulation of the robot arm with no controllers:

In [ ]:
# robot initial states [ joint 1 angle (rad),  joint 2 angle (rad), joint 1 velocity (rad/sec),  joint 1 velocity (rad/sec)]
robot.x0 = np.array([ 0.1, 0.1])

# robot constant inputs
robot.ubar = np.array([ 0.5, 1.0]) # Constant joint velocities

# run the simulation
robot.compute_trajectory( tf = 6 )

# Animate and display the simulation
ani  = robot.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

# Kinematic controller

Here bellow you can write your own custom kinematic control law, i.e. computing the joint velocity based on actual joint position and time. Here the framework is already set in place for you to speed-up the implementation process. The following variable are pre-computed and available for you to use in the feedback law:

*   The joint angles $q$
*   The time $t$

Furthermore, based on a kinematic robot model the following properties are also pre-computed:
*   The actual end-effector position $r$
*   The Jacobian $J(q)$



In [ ]:
from pyro.control  import robotcontrollers

class CustomKinematicController( robotcontrollers.EndEffectorKinematicController ) :
    def __init__(self, robot):
      super().__init__(robot)
      self.n = 0
      self.trajectory = np.array([[5, 0], [4, 0], [3, 0], [0, 3], [0, 4], [0, 5]])
    #############################
    def c( self , y , r , t ):
        """
        Feedback static computation u = c(y,r,t)

        INPUTS
        y  : sensor signal vector     p x 1
        r  : reference signal vector  k x 1
        t  : time                     1 x 1

        OUTPUTS
        dq  : joint velocity vector   m x 1

        """

        # Feedback from sensors
        q = y                  # Joint angles

        # Pre-computed values based on the robot kinematic model
        J = self.J( q )        # Jacobian computation
        r_actual = self.fwd_kin( q )  # End-effector postion

        ##############################
        # YOUR CODE BELLOW !!
        ##############################

        # Compute the reference
        # Exercice 1.1
        # if abs(r_actual[0] - self.trajectory[self.n][0]) < 0.05 and abs(r_actual[1] - self.trajectory[self.n][1]) < 0.05:
        #   self.n += 1
        #   if self.n >= 5:
        #     self.n = 5
        r_d  = np.array([
            1.5 * np.cos(0.5 * t**2),
            1.5 * np.sin(0.5 * t**2),
        ])
        e = r_d - r_actual
        dr_r = np.array([1.0, 1.0]) * e

        # Compute the desired effector velocity
        dr_d = np.array([
            -1.5 * np.sin(0.5 * t**2) * t,
            1.5 * np.cos(0.5 * t**2) * t,
        ])
        dr_r += dr_d

        # From effector speed to joint speed
        dq = np.linalg.inv( J ) @ dr_r

        return dq

In [ ]:
ctl = CustomKinematicController( robot )

## Simulating the robot in closed-loop

Here we run a simulation of the robot with the controller.

In [ ]:
# Create the closed-loop system
robot_with_controller = ctl + robot

# Run the simulation
robot_with_controller.x0[0] = 1.0
robot_with_controller.x0[1] = 1.0
robot_with_controller.compute_trajectory( tf = 12 )

In [ ]:
# Plot systems states
robot_with_controller.plot_trajectory('x')

In [ ]:
# Plot control inputs
robot_with_controller.plot_trajectory('u')

In [ ]:
# Animate the simulation
ani = robot_with_controller.generate_simulation_html_video()
html = display.HTML(ani)
display.display(html)

In [ ]:
# Exercice 2.2 - Statique d'un robot manipulateur

import matplotlib
import matplotlib.pyplot as plt

l1 = l2 = 3.0 # m
q1 = np.linspace(-np.pi, np.pi, 100) # rad
q2 = np.linspace(-np.pi, np.pi, 100) # rad
F = 1 # N

def q2torque(q1, q2) -> np.ndarray:
  t2 = F * np.sin(q1 + q2) * l2 # Nm
  t1 = F * (np.sin(q1) * l1 + np.sin(q1 + q2) * l2) # Nm
  return t1, t2

tau1 = np.zeros((np.shape(q1)[0], np.shape(q2)[0]), dtype=np.float64)
tau2 = np.zeros((np.shape(q1)[0], np.shape(q2)[0]), dtype=np.float64)
for i in range(np.shape(q1)[0]):
  for j in range(np.shape(q2)[0]):
    tau1[i, j], tau2[i, j] = q2torque(q1[i], q2[j])

# Plot the surface
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# 3. Plot the surface
# 'cmap' sets the colormap
# 'rstride' and 'cstride' control the density of the grid lines on the surface
# If you want a solid surface without grid lines, you can remove rstride/cstride or set them to 1.
surface = ax.plot_surface(q1, q1, tau1, cmap='viridis', rstride=1, cstride=1)

# 4. Add labels and a title
ax.set_xlabel('q1 (rad)')
ax.set_ylabel('q2 (rad)')
ax.set_zlabel('tau1 (Nm)')
ax.set_title('Interactive 2D Surface Plot')

# 5. Add a color bar to indicate Z values
fig.colorbar(surface, shrink=0.5, aspect=5)

# 6. Show the plot
# For interactive plots, plt.show() might block execution until the window is closed.
# With plt.ion() at the beginning, it might open the window and allow further code execution.
# In a typical script, you'd still call plt.show() at the end.
plt.show()